# Comparando audios con descriptores MFCC

**Curso**: CC5213 - Recuperación de Información Multimedia  
**Profesor**: Juan Manuel Barrios  
**Fecha**: 28 de septiembre de 2023


 Para calcular descriptores de un audio se puede usar la librería **LibROSA** con:
 
 ```
 pip install librosa
 ```

NO USAR `conda install` porque aparecerá un error por un error de versiones (al menos la versión para windows).


LibROSA puede leer archivos `wav` y retorna los samples con su samplerate con la funcion https://librosa.org/doc/main/generated/librosa.load.html:

```
    samples, sample_rate = librosa.load(archivo_wav, sr=None)
```

El argumento `sr` es el sample_rate de los samples retornados. Por defecto usa `sr=22050`  (hace una conversion). Con `sr=None` usar el mismo sample_rate del archivo wav.

Para convertir cualquier audio o video a wav usaremos FFmpeg.

Para calcular descriptores usaremos la implementación de MFCC https://librosa.org/doc/main/generated/librosa.feature.mfcc.html:

```
    matriz = librosa.feature.mfcc(y=samples, sr=sr, n_mfcc=dimension, n_fft=ventana, hop_length=salto)
    descriptores_mfcc = matriz.transpose()
```

 * `n_mfcc` es la dimensión de los descriptores a calcular (cantidad de canales a representar). Usualmente un número entre 20 y 30.
 * `n_fft` es el tamaño de ventana a representar (cantidad de samples). Idealmente debe ser potencia de 2 (e.g. 512, 1024, 2048, 4096, 8192) para que el cálculo de la Transformada de Fourier sea rápido.
 * `hop_length` es la cantidad de samples a desplazarse para calcular el siguiente descriptor. Usualmente el mismo número `n_fft` para no tener traslape o `n_fft/2` para tener un traslape de la mitad de la ventana.

Notar que LibROSA retorna **descriptores como columnas**, pero comúnmente para calcular distancias los descriptores se usan como filas, por lo que es recomendable llamar a la función `transpose()` para retornarlos como fila.

En el descriptor MFCC usualmente es buena idea descartar la primera dimensión, ya que está relacionada al volumen global del audio.

In [4]:
import numpy
import librosa
import os.path
import subprocess

def convertir_a_wav(archivo_audio, sample_rate):
    archivo_wav = "{}.{}.wav".format(archivo_audio, sample_rate) 
    print("creando: {}".format(archivo_wav))
    if os.path.isfile(archivo_wav):
        return archivo_wav
    comando = ["ffmpeg", "-i", archivo_audio, "-ac", "1", "-ar", str(sample_rate), archivo_wav]
    print("INICIANDO: {}".format(" ".join(comando)))
    code = subprocess.call(comando)
    if code != 0:
        raise Exception("ERROR!")
    return archivo_wav
    
def calcular_descriptores_mfcc(archivo_wav, samples_por_ventana, samples_salto, dimension):
    # leer audio
    samples, sr = librosa.load(archivo_wav, sr=None)
    print("audio samples={} samplerate={} segundos={:.1f}".format(len(samples), sr, len(samples) / sr))
    # calcular MFCC
    mfcc = librosa.feature.mfcc(y=samples, sr=sr, n_mfcc=dimension, n_fft=samples_por_ventana, hop_length=samples_salto)
    # convertir a descriptores por fila
    descriptores = mfcc.transpose()
    # usualmente es buena idea borrar la(s) primera(s) columna(s)
    return descriptores

def calcular_mfcc_archivo(archivo_audio, sample_rate, samples_por_ventana, samples_salto, dimension):
    archivo_wav = convertir_a_wav(archivo_audio, sample_rate)
    descriptores = calcular_descriptores_mfcc(archivo_wav, samples_por_ventana, samples_salto, dimension)
    return descriptores

archivo_audio = "vivaldi.mp3"
sample_rate = 44100          #calidad del audio (44100 es HD, se puede bajar)
samples_por_ventana = 4096   #tamaño de la ventana a la que se calcula un descriptor MFCC (usualmente unas 5 a 10 por segundo)
samples_salto = 4096         #se puede proabr con un  el salto es menor al tamaño de la ventana para que haya traslape entre ventanas
dimension = 10               #largo del descriptor MFCC (usualmente entre 10 a 64)

print("ventana={} samples ({:.0f} milisegundos)".format(samples_por_ventana, samples_por_ventana/sample_rate*1000))
print("salto  ={} samples ({:.0f} milisegundos)".format(samples_salto, samples_salto/sample_rate*1000))

descriptores_mfcc = calcular_mfcc_archivo(archivo_audio, sample_rate, samples_por_ventana, samples_salto, dimension)

print()
print("matriz de descriptores MFCC")
print("  filas={} columnas={} tipo={}".format(descriptores_mfcc.shape[0], descriptores_mfcc.shape[1], descriptores_mfcc.dtype))

descriptores_mfcc

ventana=4096 samples (93 milisegundos)
salto  =4096 samples (93 milisegundos)
creando: vivaldi.mp3.44100.wav
audio samples=3916662 samplerate=44100 segundos=88.8

matriz de descriptores MFCC
  filas=957 columnas=10 tipo=float32


array([[-3.57967438e+02,  1.05496704e+02,  1.84203815e+00, ...,
         2.57199526e+00,  4.04624653e+00,  1.28480780e+00],
       [-3.70760712e+02,  9.03441315e+01, -4.36153269e+00, ...,
         3.59831238e+00,  6.08753777e+00,  4.37231922e+00],
       [-2.93824249e+02,  1.03543488e+02, -5.23230133e+01, ...,
         2.97548699e+00, -1.89722884e+00,  1.30253077e-01],
       ...,
       [-4.15691071e+02,  6.96206360e+01,  2.63539066e+01, ...,
         1.28780651e+00,  4.95950127e+00,  6.54241657e+00],
       [-4.13459290e+02,  7.31436310e+01,  2.71731262e+01, ...,
         6.74556351e+00,  9.10780621e+00,  9.70648098e+00],
       [-3.51813385e+02,  1.03926956e+02, -6.33296728e+00, ...,
         2.95320845e+00,  4.34820414e+00,  3.35815907e+00]], dtype=float32)

# Buscar la aparición de segmentos conocidos dentro de un audio desconocido

### Primero se calculan descriptores de los audios conocidos (conjunto R)

In [ ]:
class Ventana:
    def __init__(self, nombre_archivo, segundos_desde, segundos_hasta):
        self.nombre_archivo = nombre_archivo
        self.segundos_desde = segundos_desde
        self.segundos_hasta = segundos_hasta
    
    def __str__(self):
        return "{} [{:6.3f}-{:6.3f}]".format(self.nombre_archivo, self.segundos_desde, self.segundos_hasta)

def lista_ventanas(nombre_archivo, numero_descriptores, sample_rate, samples_por_ventana):
    # tantas ventanas como numero de descriptores
    tiempos = []
    for i in range(0, samples_por_ventana * numero_descriptores, samples_por_ventana):
        # tiempo de inicio de la ventana
        segundos_desde = i / sample_rate
        # tiempo de fin de la ventana
        segundos_hasta = (i + samples_por_ventana - 1) / sample_rate
        # crear objeto
        v = Ventana(nombre_archivo, segundos_desde, segundos_hasta)
        # agregar a la lista
        tiempos.append(v)
    return tiempos


def calcular_mfcc_varios_archivos(lista_archivos, sample_rate, samples_por_ventana, samples_salto, dimension):
    descriptores_mfcc = []
    descriptores_ventanas = []
    for nombre_archivo in lista_archivos:
        audio_mfcc = calcular_mfcc_archivo(nombre_archivo, sample_rate, samples_por_ventana, samples_salto, dimension)
        audio_ventanas = lista_ventanas(nombre_archivo, audio_mfcc.shape[0], sample_rate, samples_por_ventana)
        print("  descriptores: {}".format(audio_mfcc.shape))
        if len(descriptores_mfcc) == 0:
            descriptores_mfcc = audio_mfcc
        else:
            # agregar como filas
            descriptores_mfcc = numpy.vstack([descriptores_mfcc, audio_mfcc])
        # agregar al final
        descriptores_ventanas.extend(audio_ventanas)
    return descriptores_ventanas, descriptores_mfcc
   
def imprimir_ventanas(ventanas, mfcc, muestreo_ventanas=1):
    print("ventanas={} descriptores={}".format(len(ventanas), mfcc.shape))
    print("mostrando algunas ventanas:")
    for i in range(0, len(ventanas), muestreo_ventanas):
        print(" {:4d}) {} descriptor={}".format(i, ventanas[i], mfcc[i].shape))
    
sample_rate = 44100
samples_por_ventana = 4096
samples_salto = 4096
dimension = 10
archivos_conocidos = ["vivaldi.mp3", "jaivas.mp3"]

ventanas_conocidos, mfcc_conocidos = calcular_mfcc_varios_archivos(archivos_conocidos, sample_rate, samples_por_ventana, samples_salto, dimension)

#escribir los descriptores de audios conocidos
print()
print("descriptores conocidos")
imprimir_ventanas(ventanas_conocidos, mfcc_conocidos, 50)

### Se calculan descriptores de un audio desconocido (conjunto Q)

In [ ]:
#audio consulta
query_archivo = "trozos.mp3"

ventanas_query, mfcc_query = calcular_mfcc_varios_archivos([query_archivo], sample_rate, samples_por_ventana, samples_salto, dimension)

print("Query: ventanas={} descriptores={}".format(len(ventanas_query), mfcc_query.shape))
#escribir los descriptores de audios conocidos
print()
print("descriptores audio desconocido")
imprimir_ventanas(ventanas_query, mfcc_query, 50)


### Comparar descriptores del audio desconocido (Q) con los descriptores de los audios conocidos (R)

Con cdist se comparan todos los descriptores de Q contra todos los descriptores de R y entrega la matriz de distancias.

In [ ]:
import scipy

matriz_distancias = scipy.spatial.distance.cdist(mfcc_query, mfcc_conocidos, metric='euclidean')
print(matriz_distancias.shape)
matriz_distancias

### Para cada descriptor de Q mostrar la ventana más parecida de R

Cada ventana se identifica por el nombre del archivo y el intervalo de tiempo que representa.

In [ ]:
#obtener la posicion del mas cercano por fila
posicion_min = numpy.argmin(matriz_distancias, axis=1)
minimo = numpy.amin(matriz_distancias, axis=1)

for i in range(len(ventanas_query)):
    query = ventanas_query[i]
    conocido = ventanas_conocidos[posicion_min[i]]
    diferencia = (conocido.segundos_desde - query.segundos_desde)
    print(" {:4d}) {}  se parece a  {}    (diferencia tiempos={:4.1f} seg.)".format(i, query, conocido, diferencia))

Notar que  en las zonas donde hay coincidencia de audio, los tiempos de ambas ventanas van avanzando al mismo tiempo, es decir, la diferencia entre los tiempos de sus ventanas `conocido.segundos_desde - query.segundos_desde` se mantiene constante.  

Si se agrupan las diferencias de tiempo que más se repiten, se pueden localizar las regiones comunes entre audios.

Por ejemplo, viendo la lista anterior se puede concluir que:

 * En `trozo.mp3` entre [0.186 y 5.759] se escucha el audio de `vivaldi.mp3` entre los segundos 48.019 y 53.313 (es decir, tienen un desfase de 47.6 segundos).
 * En `trozos.mp3` entre [5.851 y 8.916] se escucha `jaivas.mp3` entre [29.536 y 32.601]  (es decir, tienen un desfase de 23.7 segundos)

